<br></br>
# **자연어와 Deep Learning**
## **LSTM 단어 알파벳 완성모델**

<br></br>
## **1 Data : 학습대상**
1. Sequence Data (전체 4개 알파벳의 연속성을 학습)
1. **3개의 알파벳**으로 모델을 훈련하고, 
1. **마지막 1개를** Output과 연결하여 모델을 검증한다
1. 훈련을 위한 **자연어 Token** 은 **알파벳** 이 된다

In [ ]:
# 분석할 데이터 (원하는 다른 내용으로 변경가능) 
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

<br></br>
## **2 데이터 임배딩 객체/ 함수 (Batch 함수) 정의**
**One-Hot-Encoding** 활용

In [ ]:
# Train 에 사용할 Input / Output 데이터를 숫자로 변환하기 위한 Index 를 특정한다 
import tensorflow as tf
import numpy as np
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
num_dic  = {n: i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

print("One-Hot-Encoding 객체의 수 : {}개\nOne-Hot-Encoding 내용 :\n{}".format(
    dic_len, num_dic))

In [ ]:
# TensorFlow 에서 Token 을 숫자로 변환하는 함수
def make_batch(seq_data):
    input_batch, target_batch = [], []
    for seq in seq_data: 
        input_num = [num_dic[n] for n in seq[:-1]]  
        target    = num_dic[seq[-1]]                
        input_batch.append(np.eye(dic_len)[input_num])
        target_batch.append(target)          
    return input_batch, target_batch

<br></br>
## **3 모델의 정의**
Train 할 Tensorflow Graph 를 정의한다

In [ ]:
# 학습에 필요한 파라미터를 정의한다
tf.reset_default_graph()
n_step = 3           
learning_rate = 0.01
n_hidden, total_epoch = 64, 30
n_input = n_class = dic_len

In [ ]:
# RNN Cell 내부에서 학습하는 선형 회귀식을 정의 (tensorflow 객체)
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32,   [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class])) 
b = tf.Variable(tf.random_normal([n_class]))

In [ ]:
# RNN Cell 을 정의
# Dropout : 모든 Cell을 연결하는 경우보다 중간 중간 끊어지는 학습시, 결과가 더 잘 나옴
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)    
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)         
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

In [ ]:
# RNN Cell 의 Model을 정의한다
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model   = tf.matmul(outputs, W) + b

In [ ]:
# 학습을 보정하는 Cost 함수와, Optimizer(활성화) 함수를 정의한다
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                      logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br></br>
## **4 모델의 학습**
위에서 정의한 Model 에, **make_batch** 로 단어를 숫자로 변환 후 학습한다

In [ ]:
%%time
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, Y: target_batch})
    if epoch % 4 == 0:
        print('Epoch: {:.4f} cost = {:.6f}'.format(epoch + 1, loss))
print('최적화 모델의 학습완료!')

<br></br>
## **5 학습 모델의 평가**
seq_data 를 활용하여 모델을 검증한다

In [ ]:
%%time
# 앞에서 학습한 모델의 정확도를 판단을 위해 
# 예측 Graph 를 활성화 한다
prediction       = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)               
accuracy         = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val     = sess.run([prediction, accuracy],
                                 feed_dict={X: input_batch, Y: target_batch})

In [ ]:
# 판단 결과를 출력한다
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:-1] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)
sess.close()